# Loading data

In [124]:
import glob
import os
import numpy as np
import math
import pandas as pd
from IPython.display import display

#Reach out of IMU
#********Based on Earth coordinate system********
IMU_RO_file_path = r'D:\GoogleDrive\Graduate\Research\Research_HsinWei\Programs\data\IMUData\ReachOut'                     # use your path
IMU_RO_all_files = glob.glob(os.path.join(IMU_RO_file_path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

IMU_RO_df_list = []

for index, f in enumerate(IMU_RO_all_files):
    df = pd.read_csv(f, header=None)
    df.columns = ['x', 'y', 'z', 'timestamp', 'sampleNum']
    IMU_RO_df_list.append(df)

#Other of IMU
IMU_Other_file_path = r'D:\GoogleDrive\Graduate\Research\Research_HsinWei\Programs\data\IMUData\Other'                     # use your path
IMU_Other_all_files = glob.glob(os.path.join(IMU_Other_file_path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

IMU_Other_df_list = []

for index, f in enumerate(IMU_Other_all_files):
    df = pd.read_csv(f, header=None)
    df.columns = ['x', 'y', 'z', 'timestamp']
    IMU_Other_df_list.append(df)
    
#==============================================    

#Reach out of Kinect
#********Based on Kinect coordinate system********
Kinect_RO_file_path = r'D:\GoogleDrive\Graduate\Research\Research_HsinWei\Programs\data\KinectData\ReachOut'                     # use your path
Kinect_RO_all_files = glob.glob(os.path.join(Kinect_RO_file_path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

Kinect_RO_df_list = []

for index, f in enumerate(Kinect_RO_all_files):
    df = pd.read_csv(f, header=None)
    df.columns = ['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z',
                  'SkeletonID', 'Head_image_x', 'Head_image_y', 'timestamp', 'sampleNum']
    Kinect_RO_df_list.append(df)

#Other of Kinect
Kinect_Other_file_path = r'D:\GoogleDrive\Graduate\Research\Research_HsinWei\Programs\data\KinectData\Other'                     # use your path
Kinect_Other_all_files = glob.glob(os.path.join(Kinect_Other_file_path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

Kinect_Other_df_list = []

for index, f in enumerate(Kinect_Other_all_files):
    df = pd.read_csv(f, header=None)
    df.columns = ['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z',
                  'SkeletonID', 'Head_image_x', 'Head_image_y', 'timestamp']
    Kinect_Other_df_list.append(df)

### Reach out data

取出各個reach out 的sample段 to list that each of element is np.array

#### 1. IMU data

In [125]:
IMU_RO_sampleAll = []
# i = 0
for index, IMU_RO_raw_pd in enumerate(IMU_RO_df_list):
    sampleCount = [x for x in np.unique(IMU_RO_raw_pd['sampleNum'].values) if not math.isnan(x)]
    for count in sampleCount:
        IMU_RO_sampleAll.append(IMU_RO_raw_pd.loc[IMU_RO_raw_pd['sampleNum']==count, ['x', 'y', 'z']].values)
#         print(i, count)
#         i += 1

print("Training IMU data of reaching out have %d sample" % len(IMU_RO_sampleAll))

Training IMU data of reaching out have 65 sample


#### 2. Kinect data

In [126]:
Kinect_RO_sampleAll = []
# i = 0
for index, Kinect_RO_raw_pd in enumerate(Kinect_RO_df_list):
    sampleCount = [x for x in np.unique(Kinect_RO_raw_pd['sampleNum'].values) if not math.isnan(x)]
    for count in sampleCount:
        Kinect_RO_sampleAll.append(Kinect_RO_raw_pd.loc[Kinect_RO_raw_pd['sampleNum']==count, 
                 ['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z']].values)
#         print(i, count)
#         i += 1

print("Training Kinect data of reaching out have %d sample" % len(Kinect_RO_sampleAll))

Training Kinect data of reaching out have 144 sample


# Estimate windows size

In [127]:
# How much samples in one sec (Must need even)
sampleRate_IMU = 100
sampleRate_Kinect = 30

取最平均值↓↓↓↓↓↓↓↓↓

In [166]:
#IMU
windowSize_IMU_sa = 0
lenTemp = []
for IMU_RO_sample in IMU_RO_sampleAll:
    lenTemp.append(len(IMU_RO_sample))

windowSize_IMU_sa = np.mean(lenTemp)
windowSize_IMU = -(-(windowSize_IMU_sa)//sampleRate_IMU)
print ('IMU window: ', windowSize_IMU)

#Kinect    
windowSize_Kinect_sa = 0
lenTemp = []
for Kinect_RO_sample in Kinect_RO_sampleAll:
    lenTemp.append(len(Kinect_RO_sample))

windowSize_Kinect_sa = np.mean(lenTemp)  
windowSize_Kinect = -(-(windowSize_Kinect_sa)//sampleRate_Kinect)
print ('Kinect window: ', windowSize_Kinect)
        
#Convert to sec
windowSize_sec = max(windowSize_IMU, windowSize_Kinect)

windowSize_sec = int(windowSize_sec)

print ('Windows Size = %d' % windowSize_sec)

IMU window:  3.0
Kinect window:  2.0
Windows Size = 3


取最大值↓↓↓↓↓↓↓↓↓

In [ ]:
# #IMU
# windowSize_IMU_sa = 0
# for IMU_RO_sample in IMU_RO_sampleAll:
#     if windowSize_IMU_sa < IMU_RO_sample.shape[0]:
#         windowSize_IMU_sa = IMU_RO_sample.shape[0]
        
# windowSize_IMU = -(-(windowSize_IMU_sa)//sampleRate_IMU)
# print ('IMU window: ', windowSize_IMU)

# #Kinect    
# windowSize_Kinect_sa = 0
# for Kinect_RO_sample in Kinect_RO_sampleAll:
#     if windowSize_Kinect_sa < Kinect_RO_sample.shape[0]:
#         windowSize_Kinect_sa = Kinect_RO_sample.shape[0]

# windowSize_Kinect = -(-(windowSize_Kinect_sa)//sampleRate_Kinect)
# print ('Kinect window: ', windowSize_Kinect)
        
# #Convert to sec
# windowSize_sec = max(windowSize_IMU, windowSize_Kinect)

# windowSize_sec = int(windowSize_sec)

# # windowSize_sec = 3
# print ('Windows Size = %d' % windowSize_sec)

# Cut other data by windows size (overlapping half)

In [129]:
#IMU other data
IMU_Other_sampleAll = []
for index, IMU_Other_raw_pd in enumerate(IMU_Other_df_list):
    ranges = list(range(0, IMU_Other_raw_pd.shape[0], windowSize_sec*sampleRate_IMU//2))
    for start in ranges:
        end = start+windowSize_sec*sampleRate_IMU
        if end >= len(IMU_Other_raw_pd):
            continue
        IMU_Other_sampleAll.append(IMU_Other_raw_pd.loc[start:end-1, ['x', 'y', 'z']].values)

print("Training IMU data of Other have %d sample" % len(IMU_Other_sampleAll))

#Kinect other data
Kinect_Other_sampleAll = []
for index, Kinect_Other_raw_pd in enumerate(Kinect_Other_df_list):
    ranges = list(range(0, Kinect_Other_raw_pd.shape[0], windowSize_sec*sampleRate_Kinect//2))
    for start in ranges:
        end = start+windowSize_sec*sampleRate_Kinect
        if end >= len(Kinect_Other_raw_pd):
            continue
        Kinect_Other_sampleAll.append(Kinect_Other_raw_pd.loc[start:end-1, 
                 ['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z']].values)

print("Training Kinect data of Other have %d sample" % len(Kinect_Other_sampleAll))

Training IMU data of Other have 180 sample
Training Kinect data of Other have 114 sample


# Preprocessing

## For Kinect data, Convert coordinate to attributes that are distance & angle (Just two attributes, need more..)

#### For angle function based on Kinect coordinate system

In [130]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [131]:
from sklearn.metrics.pairwise import euclidean_distances

def conv2attr_kinect(Kinect_sample): #(N, N)
    Kinect_sample_conv = []
    for Kinect_sample_sample in Kinect_sample:
        WR = Kinect_sample_sample[0:3]
        WL = Kinect_sample_sample[3:6]
        ER = Kinect_sample_sample[6:9]
        EL = Kinect_sample_sample[9:12]
        SR = Kinect_sample_sample[12:15]
        SL = Kinect_sample_sample[15:18]
        
        WR_ER_dist = euclidean_distances([WR], [ER])[0][0]
        WR_SR_dist = euclidean_distances([WR], [SR])[0][0]
        WR_WL_dist = euclidean_distances([WR], [WL])[0][0]
        WR_EL_dist = euclidean_distances([WR], [EL])[0][0]
        WR_SL_dist = euclidean_distances([WR], [SL])[0][0]
        ER_SR_dist = euclidean_distances([ER], [SR])[0][0]
        ER_WL_dist = euclidean_distances([ER], [WL])[0][0]
        ER_EL_dist = euclidean_distances([ER], [EL])[0][0]
        ER_SL_dist = euclidean_distances([ER], [SL])[0][0]
        SR_WL_dist = euclidean_distances([SR], [WL])[0][0]
        SR_EL_dist = euclidean_distances([SR], [EL])[0][0]
        SR_SL_dist = euclidean_distances([SR], [SL])[0][0]
        WL_EL_dist = euclidean_distances([WL], [EL])[0][0]
        WL_SL_dist = euclidean_distances([WL], [SL])[0][0]
        EL_SL_dist = euclidean_distances([EL], [SL])[0][0]
        
        SR2WR_SR2SL_angle = angle_between(WR-SR, SL-SR)
        SR2ER_SR2SL_angle = angle_between(ER-SR, SL-SR)
        SL2WL_SL2SR_angle = angle_between(WL-SL, SR-SL)
        SL2EL_SL2SR_angle = angle_between(EL-SL, SR-SL)
        
        Kinect_sample_conv.append([WR_ER_dist, WR_SR_dist, WR_WL_dist, 
                           WR_EL_dist, WR_SL_dist, ER_SR_dist, 
                           ER_WL_dist, ER_EL_dist, ER_SL_dist,
                           SR_WL_dist, SR_EL_dist, SR_SL_dist, 
                           WL_EL_dist, WL_SL_dist, EL_SL_dist,
                           SR2WR_SR2SL_angle, SR2ER_SR2SL_angle, 
                           SL2WL_SL2SR_angle, SL2EL_SL2SR_angle])
    
    return np.array(Kinect_sample_conv)

# #RO
# Kinect_RO_sampleAll_conv = []
# for Kinect_RO_sample in Kinect_RO_sampleAll:
#     tempsample = []
#     for Kinect_RO_sample_sample in Kinect_RO_sample:
#         tempsample.append(euclidean_distances([Kinect_RO_sample_sample[0:3]], [Kinect_RO_sample_sample[12:15]])[0][0])
#         tempsample_np = np.array(tempsample)
#         tempsample_np = tempsample_np[:, np.newaxis]
#     Kinect_RO_sampleAll_conv.append(tempsample_np)
    
# #Other
# Kinect_Other_sampleAll_conv = []
# for Kinect_Other_sample in Kinect_Other_sampleAll:
#     tempsample = []
#     for Kinect_Other_sample_sample in Kinect_Other_sample:
#         tempsample.append(euclidean_distances([Kinect_Other_sample_sample[0:3]], [Kinect_Other_sample_sample[12:15]])[0][0])
#         tempsample_np = np.array(tempsample)
#         tempsample_np = tempsample_np[:, np.newaxis]
#     Kinect_Other_sampleAll_conv.append(tempsample_np)
    
Kinect_RO_sampleAll_conv = []

for Kinect_RO_sample in Kinect_RO_sampleAll:
    Kinect_RO_sampleAll_conv.append(conv2attr_kinect(Kinect_RO_sample))
    
Kinect_Other_sampleAll_conv = []

for Kinect_Other_sample in Kinect_Other_sampleAll:
    Kinect_Other_sampleAll_conv.append(conv2attr_kinect(Kinect_Other_sample))
    
print('Number of Kinect\'s attribute:', len(Kinect_RO_sampleAll_conv[0][0]))

Number of Kinect's attribute: 19


# Feature extraction

### Feature of IMU

In [168]:
from scipy.stats import skew
from scipy.stats import kurtosis

def extract_feature_IMU(IMU_sample): #input np.array shape = (N, 3)

    ##Mean (1, 3)
    IMU_mean = np.mean(IMU_sample, axis=0) 
    
    ##Skewness
    IMU_skew = skew(IMU_sample, axis=0)
    
    ##Kurtosis
    IMU_kurtosis = kurtosis(IMU_sample, axis=0)
    
    ##Mean Magnitude (1,)
    summ = 0.0
    for IMU_sample_sample in IMU_sample:
        tempSqur = 0.0
        for i in range(0, len(IMU_sample_sample)):
            tempSqur += IMU_sample_sample[i]**2
        summ = tempSqur**0.5
    IMU_MM = (summ/IMU_sample.shape[0])

    ##Variance (1, 3)
    IMU_var = np.var(IMU_sample, axis=0) 

    ##Median (1, 3)
    IMU_median = np.median(IMU_sample, axis=0)
    
    #-------------------------------------------
    #
    
    #-------------------------------------------
    #Time feature

    ##Mean of 1 sec (1, 3*windowSize)...
    #feature count is windowsSize/sec. 
    #Each of mean of reaching out are 100 samples.
    oneSample = []
    for start in range(0, windowSize_sec*sampleRate_IMU, sampleRate_IMU):
        end = start + sampleRate_IMU
        tempSamples = IMU_sample[start:end]
        oneSample.extend(np.mean(tempSamples, axis=0).tolist())
    IMU_meanOf1sec = oneSample

    #combine together
    features = []
    features.extend(IMU_mean.tolist())
    features.extend(IMU_skew.tolist())
    features.extend(IMU_kurtosis.tolist())
    features.append(IMU_MM)
    features.extend(IMU_var.tolist())
    features.extend(IMU_median.tolist())
    features.extend(IMU_meanOf1sec)
    
    return features

### Feature of Kinect

In [169]:
def extract_feature_Kinect(Kinect_sample): #input np.array shape = (N, ?)
    ##Mean
    Kinect_mean = np.mean(Kinect_sample, axis=0)
    
    ##Skewness
    Kinect_skew = skew(Kinect_sample, axis=0)
    
    ##Kurtosis
    Kinect_kurtosis = kurtosis(Kinect_sample, axis=0)

    ##Mean Magnitude
    summ = 0.0
    for Kinect_sample_sample in Kinect_sample:
        tempSqur = 0.0
        for i in range(0, len(Kinect_sample_sample)):
            tempSqur += Kinect_sample_sample[i]**2
        summ = tempSqur**0.5
    Kinect_MM = (summ/Kinect_sample.shape[0])
    
    ##Variance
    Kinect_var = np.var(Kinect_sample, axis=0)

    ##Median
    Kinect_median = np.median(Kinect_sample, axis=0)

    ##Mean of 1 sec
    #feature count is windowsSize/sec. 
    #Each of mean of reaching out are 24 samples. 不夠的設為0 
    oneSample = []
    for start in range(0, windowSize_sec*sampleRate_Kinect, sampleRate_Kinect):
        end = start + sampleRate_Kinect
        tempSamples = Kinect_sample[start:end]
        oneSample.extend(np.mean(tempSamples, axis=0).tolist())
    Kinect_meanOf1sec = oneSample
    
    #combine together
    features = []
    features.extend(Kinect_mean.tolist())
    features.extend(Kinect_skew.tolist())
    features.extend(Kinect_kurtosis.tolist())
    features.append(Kinect_MM)
    features.extend(Kinect_var.tolist())
    features.extend(Kinect_median.tolist())
    features.extend(Kinect_meanOf1sec)
    
    return features

## 補足data長度到windows size(前後補)&Split data into Training, Validation and Testing(6/2/2) & Standard

#### IMU

In [170]:
sampleRate_IMU*windowSize_sec-len(IMU_RO_sample)

61

In [171]:
X_IMU = []
y_IMU = []

#add RO into X_IMU & y_IMU
for IMU_RO_sample in IMU_RO_sampleAll:
    lengthOflack = sampleRate_IMU*windowSize_sec-len(IMU_RO_sample)
    if lengthOflack > 0:
        IMU_RO_sample_filltoprior = np.concatenate( 
                                  ([IMU_RO_sample[0].tolist()] * lengthOflack, IMU_RO_sample),
                                  axis=0)
        IMU_RO_sample_filltoPandR = np.concatenate(
                                  ([IMU_RO_sample[0].tolist()] * (lengthOflack//2), IMU_RO_sample, [IMU_RO_sample[-1].tolist()] * (-(-lengthOflack//2))), 
                                  axis=0)
        IMU_RO_sample_filltorear = np.concatenate(
                                  (IMU_RO_sample, [IMU_RO_sample[-1].tolist()] * lengthOflack), 
                                  axis=0)
    else:
        IMU_RO_sample_filltoprior = IMU_RO_sample
        IMU_RO_sample_filltoPandR = IMU_RO_sample
        IMU_RO_sample_filltorear = IMU_RO_sample
    features_prior = extract_feature_IMU(IMU_RO_sample_filltoprior)
    features_PandR = extract_feature_IMU(IMU_RO_sample_filltoPandR)
    features_rear = extract_feature_IMU(IMU_RO_sample_filltorear)
    #add into X_IMU
    X_IMU.append(features_prior)
    y_IMU.append(1)
    X_IMU.append(features_PandR)
    y_IMU.append(1)
    X_IMU.append(features_rear)
    y_IMU.append(1)

#add Other into X_IMU & y_IMU
for IMU_Other_sample in IMU_Other_sampleAll:
    features = extract_feature_IMU(IMU_Other_sample)
    #add into X
    X_IMU.append(features)
    y_IMU.append(0)
    

##Split X_IMU, y_IMU into training and testing(6/2/2)
from sklearn.model_selection import train_test_split

# X_IMU_train, X_IMU_test, y_IMU_train, y_IMU_test = train_test_split(X_IMU, y_IMU, test_size=0.2, random_state=0)
# X_IMU_train, X_IMU_val, y_IMU_train, y_IMU_val = train_test_split(X_IMU_train, y_IMU_train, test_size=0.25, random_state=0)
X_IMU_train, X_IMU_val, y_IMU_train, y_IMU_val = train_test_split(X_IMU, y_IMU, test_size=0.3, random_state=0)


#Standard
from sklearn.preprocessing import StandardScaler

sc_IMU = StandardScaler()
sc_IMU.fit(X_IMU_train)
X_IMU_train_std = sc_IMU.transform(X_IMU_train)
X_IMU_val_std = sc_IMU.transform(X_IMU_val)
# X_IMU_test_std = sc_IMU.transform(X_IMU_test)

#### Kinect

In [172]:
X_Kinect = []
y_Kinect = []

#add RO into X_Kinect & y_Kinect
for Kinect_RO_sample_conv in Kinect_RO_sampleAll_conv:
    lengthOflack = sampleRate_Kinect*windowSize_sec-len(Kinect_RO_sample_conv)
    if lengthOflack > 0:
        Kinect_RO_sample_filltoprior = np.concatenate( 
                                  ([Kinect_RO_sample_conv[0].tolist()] * lengthOflack, Kinect_RO_sample_conv),
                                  axis=0)
        Kinect_RO_sample_filltoPandR = np.concatenate(
                                  ([Kinect_RO_sample_conv[0].tolist()] * (lengthOflack//2), Kinect_RO_sample_conv, [Kinect_RO_sample_conv[-1].tolist()] * (-(-lengthOflack//2))), 
                                  axis=0)
        Kinect_RO_sample_filltorear = np.concatenate(
                                  (Kinect_RO_sample_conv, [Kinect_RO_sample_conv[-1].tolist()] * lengthOflack), 
                                  axis=0)
    else:
        Kinect_RO_sample_filltoprior = Kinect_RO_sample_conv
        Kinect_RO_sample_filltoPandR = Kinect_RO_sample_conv
        Kinect_RO_sample_filltorear = Kinect_RO_sample_conv
    features_prior = extract_feature_Kinect(Kinect_RO_sample_filltoprior)
    features_PandR = extract_feature_Kinect(Kinect_RO_sample_filltoPandR)
    features_rear = extract_feature_Kinect(Kinect_RO_sample_filltorear)
    #add into X_Kinect
    X_Kinect.append(features_prior)
    y_Kinect.append(1)
    X_Kinect.append(features_PandR)
    y_Kinect.append(1)
    X_Kinect.append(features_rear)
    y_Kinect.append(1)

#add Other into X_Kinect & y_Kinect
for Kinect_Other_sample_conv in Kinect_Other_sampleAll_conv:
    features = extract_feature_Kinect(Kinect_Other_sample_conv)
    #add into X_Kinect
    X_Kinect.append(features)
    y_Kinect.append(0)
    

##Split X_Kinect, y_Kinect into training and testing(6/2/2)
from sklearn.model_selection import train_test_split

# X_Kinect_train, X_Kinect_test, y_Kinect_train, y_Kinect_test = train_test_split(X_Kinect, y_Kinect, test_size=0.2, random_state=2)
# X_Kinect_train, X_Kinect_val, y_Kinect_train, y_Kinect_val = train_test_split(X_Kinect_train, y_Kinect_train, test_size=0.25, random_state=2)
X_Kinect_train, X_Kinect_val, y_Kinect_train, y_Kinect_val = train_test_split(X_Kinect, y_Kinect, test_size=0.3, random_state=2)


#Standard
from sklearn.preprocessing import StandardScaler

sc_Kinect = StandardScaler()
sc_Kinect.fit(X_Kinect_train)
X_Kinect_train_std = sc_Kinect.transform(X_Kinect_train)
X_Kinect_val_std = sc_Kinect.transform(X_Kinect_val)
# X_Kinect_test_std = sc_Kinect.transform(X_Kinect_test)

In [173]:
print('Training data的比率 of IMU')
uni ,count = np.unique(y_IMU_train, return_counts=True)
print (dict(zip(uni, count)))

print('Training data的比率 of Kinect')
uni ,count = np.unique(y_Kinect_train, return_counts=True)
print (dict(zip(uni, count)))

Training data的比率 of IMU
{0: 125, 1: 137}
Training data的比率 of Kinect
{0: 81, 1: 301}


# 丟入model 看結果(緊張..)

#### Decision Tree

In [174]:
print('======Decision Tree=======')
from sklearn.tree import DecisionTreeClassifier

# criterion : impurity function
# max_depth : maximum depth of tree
# random_state : seed of random number generator
##IMU
tree_IMU = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=7, 
                              random_state=0)
tree_IMU.fit(X_IMU_train_std, y_IMU_train)

print('[IMU performance]')
#Validation
from sklearn.metrics import accuracy_score
y_IMU_pred = tree_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = tree_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##Kinect
tree_Kinect = DecisionTreeClassifier(criterion='gini', 
                                     splitter='random',
                                     max_depth=None,
                                     min_samples_split=12 ,
                                     random_state=5)
tree_Kinect.fit(X_Kinect_train_std, y_Kinect_train)

print('\n[Kinect performance]')
#Validation
y_Kinect_pred = tree_Kinect.predict(X_Kinect_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_Kinect_val, y_Kinect_pred))
#Testing
# y_Kinect_pred = tree_Kinect.predict(X_Kinect_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_Kinect_test, y_Kinect_pred))

#Persist model
from sklearn.externals import joblib

joblib.dump(tree_IMU, './models/tree_IMU.pkl')
joblib.dump(tree_Kinect, './models/tree_Kinect.pkl')

======Decision Tree=======
[IMU performance]
Validation accuracy: 0.94

[Kinect performance]
Validation accuracy: 0.96


['./models/tree_Kinect.pkl']

#### RandomForest

In [175]:
print('======Random Forest=======')
from sklearn.ensemble import RandomForestClassifier

# criterion : impurity function
# n_estimators :  number of decision trees
# random_state : seed used by the random number generator
# n_jobs : number of cores for parallelism
#IMU
forest_IMU = RandomForestClassifier(criterion='entropy',
                                max_depth=7,
                                n_estimators=200, 
                                random_state=1,
                                n_jobs=-1)
forest_IMU.fit(X_IMU_train_std, y_IMU_train)

print('[IMU performance]')
#Validation
y_IMU_pred = forest_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = forest_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##Kinect
forest_Kinect = RandomForestClassifier(criterion='entropy',
                                max_depth=7,
                                n_estimators=300, 
                                random_state=1,
                                n_jobs=-1)
forest_Kinect.fit(X_Kinect_train_std, y_Kinect_train)

print('\n[Kinect performance]')
#Validation
y_Kinect_pred = forest_Kinect.predict(X_Kinect_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_Kinect_val, y_Kinect_pred))
#Testing
# y_Kinect_pred = forest_Kinect.predict(X_Kinect_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_Kinect_test, y_Kinect_pred))

#Persist model
joblib.dump(forest_IMU, './models/forest_IMU.pkl')
joblib.dump(forest_Kinect, './models/forest_Kinect.pkl')

======Random Forest=======
[IMU performance]
Validation accuracy: 0.96

[Kinect performance]
Validation accuracy: 0.99


['./models/forest_Kinect.pkl']

#### KNN

In [176]:
print('======KNN=======')
from sklearn.neighbors import KNeighborsClassifier

# p=2 and metric='minkowski' means the Euclidean Distance
##IMU
knn_IMU = KNeighborsClassifier(n_neighbors=3, p=2, metric='minkowski')

knn_IMU.fit(X_IMU_train_std, y_IMU_train)

print('[IMU performance]')
#Validation
y_IMU_pred = knn_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = knn_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##Kinect
knn_Kinect = KNeighborsClassifier(n_neighbors=2, p=1, metric='minkowski')

knn_Kinect.fit(X_Kinect_train_std, y_Kinect_train)

print('\n[Kinect performance]')
#Validation
y_Kinect_pred = knn_Kinect.predict(X_Kinect_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_Kinect_val, y_Kinect_pred))
#Testing
# y_Kinect_pred = knn_Kinect.predict(X_Kinect_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_Kinect_test, y_Kinect_pred))

#Persist model
joblib.dump(knn_IMU, './models/knn_IMU.pkl')
joblib.dump(knn_Kinect, './models/knn_Kinect.pkl')

======KNN=======
[IMU performance]
Validation accuracy: 0.99

[Kinect performance]
Validation accuracy: 0.98


['./models/knn_Kinect.pkl']

#### SVM

In [177]:
print('=====SVM=======')
from sklearn.svm import SVC

# kernel: the kernel function, can be 'linear', 'poly', 'rbf', ...etc
# C is the hyperparameter for the error penalty term
##IMU
svm_linear_IMU = SVC(kernel='linear', C=10000.0, random_state=0)

svm_linear_IMU.fit(X_IMU_train_std, y_IMU_train)

print('#[IMU performance]#')
#Validation
print('[Linear SVC]')
y_IMU_pred = svm_linear_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = svm_linear_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##poly
svm_poly_IMU = SVC(kernel='poly', C=1000.0, random_state=0)

svm_poly_IMU.fit(X_IMU_train_std, y_IMU_train)

#Validation
print('[Poly SVC]')
y_IMU_pred = svm_poly_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = svm_poly_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##rbf
svm_rbf_IMU = SVC(kernel='rbf', C=10000.0, random_state=0)

svm_rbf_IMU.fit(X_IMU_train_std, y_IMU_train)

#Validation
print('[rbf SVC]')
y_IMU_pred = svm_rbf_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = svm_rbf_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##Kinect
svm_linear_Kinect = SVC(kernel='linear', C=10000.0, random_state=0)

svm_linear_Kinect.fit(X_Kinect_train_std, y_Kinect_train)

print('\n#[Kinect performance]#')
#Validation
print('[Linear SVC]')
y_Kinect_pred = svm_linear_Kinect.predict(X_Kinect_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_Kinect_val, y_Kinect_pred))
#Testing
# y_Kinect_pred = svm_linear_Kinect.predict(X_Kinect_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_Kinect_test, y_Kinect_pred))

##poly
svm_poly_Kinect = SVC(kernel='poly', C=1000.0, random_state=0)

svm_poly_Kinect.fit(X_Kinect_train_std, y_Kinect_train)

#Validation
print('[Poly SVC]')
y_Kinect_pred = svm_poly_Kinect.predict(X_Kinect_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_Kinect_val, y_Kinect_pred))
#Testing
# y_Kinect_pred = svm_poly_Kinect.predict(X_Kinect_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_Kinect_test, y_Kinect_pred))

##rbf
svm_rbf_Kinect = SVC(kernel='rbf', C=10000.0, random_state=0)

svm_rbf_Kinect.fit(X_Kinect_train_std, y_Kinect_train)

#Validation
print('[rbf SVC]')
y_Kinect_pred = svm_rbf_Kinect.predict(X_Kinect_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_Kinect_val, y_Kinect_pred))
#Testing
# y_Kinect_pred = svm_rbf_Kinect.predict(X_Kinect_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_Kinect_test, y_Kinect_pred))


#Persist model
joblib.dump(svm_linear_IMU, './models/svm_linear_IMU.pkl')
joblib.dump(svm_poly_IMU, './models/svm_poly_IMU.pkl')
joblib.dump(svm_rbf_IMU, './models/svm_rbf_IMU.pkl')
joblib.dump(svm_linear_Kinect, './models/svm_linear_Kinect.pkl')
joblib.dump(svm_poly_Kinect, './models/svm_poly_Kinect.pkl')
joblib.dump(svm_rbf_Kinect, './models/svm_rbf_Kinect.pkl')

=====SVM=======
#[IMU performance]#
[Linear SVC]
Validation accuracy: 0.94
[Poly SVC]
Validation accuracy: 0.99
[rbf SVC]
Validation accuracy: 0.99

#[Kinect performance]#
[Linear SVC]
Validation accuracy: 0.98
[Poly SVC]
Validation accuracy: 0.92
[rbf SVC]
Validation accuracy: 1.00


['./models/svm_rbf_Kinect.pkl']

#### Neural network

In [178]:
print('======Neural network=======')
from sklearn.neural_network import MLPClassifier

NN_IMU = MLPClassifier(solver='lbfgs', alpha=0.01, activation='logistic', learning_rate='constant', learning_rate_init=0.001,
                    hidden_layer_sizes=(5, 2), random_state=1)

NN_IMU.fit(X_IMU_train_std, y_IMU_train)

print('#[IMU performance]#')
#Validation
y_IMU_pred = NN_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))

##Kinect
NN_Kinect = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

NN_Kinect.fit(X_Kinect_train_std, y_Kinect_train)
print('#[Kinect performance]#')
#Validation
y_IMU_pred = NN_Kinect.predict(X_Kinect_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_Kinect_val, y_Kinect_pred))

#Persist model
joblib.dump(NN_IMU, './models/NN_IMU.pkl')
joblib.dump(NN_Kinect, './models/NN_Kinect.pkl')

======Neural network=======
#[IMU performance]#
Validation accuracy: 0.99
#[Kinect performance]#
Validation accuracy: 1.00


['./models/NN_Kinect.pkl']

# Ensemble methods

#### Voting

In [44]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
import itertools

best_vt, best_w, best_score = None, (), -1
for a, b, c in list(itertools.permutations(range(0, 3))):  # try some weight combination
    voting_IMU = VotingClassifier(
                        estimators=[('tree', tree_IMU), ('forest', forest_IMU), ('knn', knn_IMU)],
                          voting='soft',
                          weights=[a, b, c])
    scores = cross_val_score(
        estimator=voting_IMU, X=X_IMU_train_std, y=y_IMU_train, cv=10, scoring='roc_auc')
    print('%s: %.3f (+/- %.3f)' % ((a, b, c), scores.mean(), scores.std()))
    if best_score < scores.mean():
        best_vt, best_w, best_score = voting_IMU, (a, b, c), scores.mean()

print('\nBest %s: %.3f' % (best_w, best_score))

(0, 1, 2): 1.000 (+/- 0.000)
(0, 2, 1): 1.000 (+/- 0.000)
(1, 0, 2): 1.000 (+/- 0.000)
(1, 2, 0): 0.998 (+/- 0.005)
(2, 0, 1): 0.998 (+/- 0.005)
(2, 1, 0): 0.997 (+/- 0.008)

Best (0, 1, 2): 1.000


# Simulating real world

### IMU

In [144]:
import random

X_combined_IMU_All = []
y_combined_IMU_All = []

##先給0~100 Other then replace RO in random place
for random_Other_sample in random.sample(IMU_Other_sampleAll, 30):
    X_combined_IMU_All.append(random_Other_sample)
    y_combined_IMU_All.append(0)

insertROcount = 6
for index, random_RO_sample in zip(np.random.randint(30, size=insertROcount), random.sample(IMU_RO_sampleAll, insertROcount)):
    X_combined_IMU_All[index] = random_RO_sample
    y_combined_IMU_All[index] = 1
    
##flatten
X_combined_IMU_fla = []
y_combined_IMU_fla = []

for s, c in zip(X_combined_IMU_All, y_combined_IMU_All):
    X_combined_IMU_fla.extend(s.tolist())
    listofzerosOrOne = [c] * s.shape[0]
    y_combined_IMU_fla.extend(listofzerosOrOne)
    
##Segmentation(by windows Size, and Overlapping= windows size - 1 sec)
overlapping = windowSize_sec - 1
X_combined_IMU_seg = []

for start in range(0, len(X_combined_IMU_fla), windowSize_sec*sampleRate_IMU - overlapping*sampleRate_IMU):
    end = start + sampleRate_IMU*windowSize_sec
    if len(X_combined_IMU_fla[start:end]) < sampleRate_IMU*windowSize_sec:
        continue
    else:
        X_combined_IMU_seg.append(X_combined_IMU_fla[start:end])
        
# ##feature extraction
X_combined_IMU = []

for X_combined_IMU_seg_sample in X_combined_IMU_seg:
    X_combined_IMU.append(extract_feature_IMU(np.array(X_combined_IMU_seg_sample)))
        
##Normalization
X_combined_IMU_std = sc_IMU.transform(X_combined_IMU)

### Kinect

In [145]:
X_combined_Kinect_All = []
y_combined_Kinect_All = []

##先給0~100 Other then replace RO in random place
for random_Other_sample in random.sample(Kinect_Other_sampleAll, 30):
    X_combined_Kinect_All.append(random_Other_sample)
    y_combined_Kinect_All.append(0)

insertROcount = 6
for index, random_RO_sample in zip(np.random.randint(30, size=insertROcount), random.sample(Kinect_RO_sampleAll, insertROcount)):
    X_combined_Kinect_All[index] = random_RO_sample
    y_combined_Kinect_All[index] = 1
    
##flatten
X_combined_Kinect_fla = []
y_combined_Kinect_fla = []

for s, c in zip(X_combined_Kinect_All, y_combined_Kinect_All):
    X_combined_Kinect_fla.extend(s.tolist())
    listofzerosOrOne = [c] * s.shape[0]
    y_combined_Kinect_fla.extend(listofzerosOrOne)
    
##Segmentation(by windows Size, and Overlapping= windows size - 1 sec)
overlapping = windowSize_sec - 1
X_combined_Kinect_seg = []

for start in range(0, len(X_combined_Kinect_fla), windowSize_sec*sampleRate_Kinect - overlapping*sampleRate_Kinect):
    end = start + sampleRate_Kinect*windowSize_sec
    if len(X_combined_Kinect_fla[start:end]) < sampleRate_Kinect*windowSize_sec:
        continue
    else:
        X_combined_Kinect_seg.append(X_combined_Kinect_fla[start:end])

#Preprocessing
X_combined_Kinect_seg_conv = []
for X_combined_Kinect_seg_sam in X_combined_Kinect_seg:
    X_combined_Kinect_seg_conv.append(conv2attr_kinect(np.array(X_combined_Kinect_seg_sam)))
        
##feature extraction
X_combined_Kinect = []

for X_combined_Kinect_seg_sample in X_combined_Kinect_seg_conv:
    X_combined_Kinect.append(extract_feature_Kinect(X_combined_Kinect_seg_sample))
        
##Normalization
X_combined_Kinect_std = sc_Kinect.transform(X_combined_Kinect)

## Input to models

In [146]:
#將Pred結果展開成sample比較 #Overlapping 實際執行秒數 = (Sec*Num)-[(Num-1)*Overlap]
def pred_flatToSamples_IMU(y_pred):
    allofSec = windowSize_sec*len(y_pred)-(len(y_pred)-1)*overlapping
    y_pred_sample = [0] * (allofSec*sampleRate_IMU)
    for pred, index in zip(y_pred, range(0, allofSec, windowSize_sec-overlapping)):
        if pred == 1:
            start = index*sampleRate_IMU
            end = start + sampleRate_IMU*windowSize_sec
            listofOne = [1] * (sampleRate_IMU*windowSize_sec)
            y_pred_sample[start:end] = listofOne
    return y_pred_sample

def pred_flatToSamples_Kinect(y_pred):
    allofSec = windowSize_sec*len(y_pred)-(len(y_pred)-1)*overlapping
    y_pred_sample = [0] * (allofSec*sampleRate_Kinect)
    for pred, index in zip(y_pred, range(0, allofSec, windowSize_sec-overlapping)):
        if pred == 1:
            start = index*sampleRate_Kinect
            end = start + sampleRate_Kinect*windowSize_sec
            listofOne = [1] * (sampleRate_Kinect*windowSize_sec)
            y_pred_sample[start:end] = listofOne
    return y_pred_sample

In [147]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
# %pylab inline

print('--IMU--')
tree_IMU = joblib.load('./models/tree_IMU.pkl')
y_pred = tree_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

# plot(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis])
# plot(np.array(y_pred_sample)[:,np.newaxis])
dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('Decision tree \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

forest_IMU = joblib.load('./models/forest_IMU.pkl')
y_pred = forest_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('Random forest \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

knn_IMU = joblib.load('./models/knn_IMU.pkl')
y_pred = knn_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('KNN \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

svm_linear_IMU = joblib.load('./models/svm_linear_IMU.pkl')
y_pred = svm_linear_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM linear \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

svm_poly_IMU = joblib.load('./models/svm_poly_IMU.pkl')
y_pred = svm_poly_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM poly \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

svm_rbf_IMU = joblib.load('./models/svm_rbf_IMU.pkl')
y_pred = svm_rbf_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM rbf \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

NN_IMU = joblib.load('./models/NN_IMU.pkl')
y_pred = NN_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('NN \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

##Kinect
print('--Kinect--')
tree_Kinect = joblib.load('./models/tree_Kinect.pkl')
y_pred = tree_Kinect.predict(X_combined_Kinect_std)
y_pred_sample = pred_flatToSamples_Kinect(y_pred)

dist, path = fastdtw(np.array(y_combined_Kinect_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('Decistion tree \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_Kinect_fla[:len(y_pred_sample)], y_pred_sample))

forest_Kinect = joblib.load('./models/forest_Kinect.pkl')
y_pred = forest_Kinect.predict(X_combined_Kinect_std)
y_pred_sample = pred_flatToSamples_Kinect(y_pred)

dist, path = fastdtw(np.array(y_combined_Kinect_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('Random forest \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_Kinect_fla[:len(y_pred_sample)], y_pred_sample))

knn_Kinect = joblib.load('./models/knn_Kinect.pkl')
y_pred = knn_Kinect.predict(X_combined_Kinect_std)
y_pred_sample = pred_flatToSamples_Kinect(y_pred)

dist, path = fastdtw(np.array(y_combined_Kinect_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('KNN \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_Kinect_fla[:len(y_pred_sample)], y_pred_sample))

svm_linear_Kinect = joblib.load('./models/svm_linear_Kinect.pkl')
y_pred = svm_linear_Kinect.predict(X_combined_Kinect_std)
y_pred_sample = pred_flatToSamples_Kinect(y_pred)

dist, path = fastdtw(np.array(y_combined_Kinect_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM linear \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_Kinect_fla[:len(y_pred_sample)], y_pred_sample))

svm_poly_Kinect = joblib.load('./models/svm_poly_Kinect.pkl')
y_pred = svm_poly_Kinect.predict(X_combined_Kinect_std)
y_pred_sample = pred_flatToSamples_Kinect(y_pred)

dist, path = fastdtw(np.array(y_combined_Kinect_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM poly \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_Kinect_fla[:len(y_pred_sample)], y_pred_sample))

svm_rbf_Kinect = joblib.load('./models/svm_rbf_Kinect.pkl')
y_pred = svm_rbf_Kinect.predict(X_combined_Kinect_std)
y_pred_sample = pred_flatToSamples_Kinect(y_pred)

dist, path = fastdtw(np.array(y_combined_Kinect_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM rbf \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_Kinect_fla[:len(y_pred_sample)], y_pred_sample))

NN_Kinect = joblib.load('./models/NN_Kinect.pkl')
y_pred = NN_Kinect.predict(X_combined_Kinect_std)
y_pred_sample = pred_flatToSamples_Kinect(y_pred)

dist, path = fastdtw(np.array(y_combined_Kinect_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('NN \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_Kinect_fla[:len(y_pred_sample)], y_pred_sample))

--IMU--
Decision tree 
-DTW: 1343.0
-Accuracy: 0.69
Random forest 
-DTW: 373.0
-Accuracy: 0.90
KNN 
-DTW: 343.0
-Accuracy: 0.75
SVM linear 
-DTW: 590.0
-Accuracy: 0.79
SVM poly 
-DTW: 533.0
-Accuracy: 0.90
SVM rbf 
-DTW: 433.0
-Accuracy: 0.82
NN 
-DTW: 633.0
-Accuracy: 0.82
--Kinect--
-Decistion tree 
-DTW: 0.0
-Accuracy: 0.91
-Random forest 
-DTW: 39.0
-Accuracy: 0.94
-KNN 
-DTW: 0.0
-Accuracy: 0.91
-SVM linear 
-DTW: 0.0
-Accuracy: 0.93
-SVM poly 
-DTW: 360.0
-Accuracy: 0.66
-SVM rbf 
-DTW: 51.0
-Accuracy: 0.95
-NN 
-DTW: 39.0
-Accuracy: 0.96


In [ ]:
1/0

# Experiencing real data

IMU

In [179]:
IMU_rW_pd = pd.read_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/realWorldData/IMU/raw_acc_01.csv', header=None)
# IMU_rW_pd = pd.read_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/IMUData/ReachOut/HW_raw_acc_RO_02.csv', header=None)

IMU_rW_pd.columns = ['x', 'y', 'z', 'timestamp', 'ReachOut']

IMU_rW = IMU_rW_pd[['x', 'y', 'z']].values.tolist()

##Segmentation(by windows Size, and Overlapping= windows size - 1 sec)
overlapping = windowSize_sec - 1
X_rW_IMU_seg = []

for start in range(0, len(IMU_rW), windowSize_sec*sampleRate_IMU - overlapping*sampleRate_IMU):
    end = start + sampleRate_IMU*windowSize_sec
    if len(IMU_rW[start:end]) < sampleRate_IMU*windowSize_sec:
        break
    else:
        X_rW_IMU_seg.append(IMU_rW[start:end])
        
#Preprocessing
X_rW_IMU_seg_conv = []
for X_rW_IMU_seg_sam in X_rW_IMU_seg:
    X_rW_IMU_seg_conv.append(X_rW_IMU_seg_sam)
        
##feature extraction
X_rW_IMU = []
for X_rW_IMU_seg_sample in X_rW_IMU_seg_conv:
    X_rW_IMU.append(extract_feature_IMU(np.array(X_rW_IMU_seg_sample)))
        
##Normalization
X_rW_IMU_std = sc_IMU.transform(X_rW_IMU)

Kinect

In [180]:
Kinect_rW_pd = pd.read_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/realWorldData/Kinect/VSFile_01.csv', header=None)
Kinect_rW_pd.columns = ['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z',
                  'SkeletonID', 'Head_image_x', 'Head_image_y', 'timestamp']

Kinect_rW_pd_ntime = Kinect_rW_pd[['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z',
                  'SkeletonID', 'Head_image_x', 'Head_image_y']]

Kinect_rW = Kinect_rW_pd_ntime.values.tolist()

##Segmentation(by windows Size, and Overlapping= windows size - 1 sec)
overlapping = windowSize_sec - 1
X_rW_Kinect_seg = []

for start in range(0, len(Kinect_rW), windowSize_sec*sampleRate_Kinect - overlapping*sampleRate_Kinect):
    end = start + sampleRate_Kinect*windowSize_sec
    if len(Kinect_rW[start:end]) < sampleRate_Kinect*windowSize_sec:
        continue
    else:
        X_rW_Kinect_seg.append(Kinect_rW[start:end])
        
#Preprocessing
X_rW_Kinect_seg_conv = []
for X_rW_Kinect_seg_sam in X_rW_Kinect_seg:
    X_rW_Kinect_seg_conv.append(conv2attr_kinect(np.array(X_rW_Kinect_seg_sam)))
        
##feature extraction
X_rW_Kinect = []

for X_rW_Kinect_seg_sample in X_rW_Kinect_seg_conv:
    X_rW_Kinect.append(extract_feature_Kinect(X_rW_Kinect_seg_sample))
        
##Normalization
X_rW_Kinect_std = sc_Kinect.transform(X_rW_Kinect)

### Feed into models

In [181]:
svm_rbf_Kinect = joblib.load('./models/svm_rbf_Kinect.pkl')
forest_Kinect.predict(X_rW_Kinect_std)

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [182]:
svm_linear_IMU.predict(X_rW_IMU_std)

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [188]:
svm_linear_Kinect.predict(X_rW_Kinect_std)

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [192]:
NN_IMU.predict(X_rW_IMU_std)

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [185]:
NN_Kinect.predict(X_rW_Kinect_std)

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [196]:
y_IMU_pred = NN_IMU.predict(X_rW_IMU_std)

y_kinect_pred = NN_Kinect.predict(X_rW_Kinect_std)


IMU_sample_rw_pred = pred_flatToSamples_IMU(y_IMU_pred)
Kinect_sample_rw_pred = pred_flatToSamples_Kinect(y_kinect_pred)

# #Aligning
# from datetime import datetime
# IMU_rW_time = IMU_rW_pd['timestamp']
# Kinect_rW_time = Kinect_rW_pd['timestamp']

# startIndex_IMU = 0
# start_Kinect = datetime.strptime(Kinect_rW_time[0], '%H:%M:%S:%f')
# IMU_rW_time = pd.to_datetime(IMU_rW_time, format='%H:%M:%S:%f')
# for index, IMU_timestamp in zip(range(0, len(IMU_rW_time)), IMU_rW_time):
#     if IMU_timestamp < start_Kinect:
#         continue
#     else:
#         startIndex_IMU = index
#         break
        
# IMU_sample_rw_pred = IMU_sample_rw_pred[startIndex_IMU:]

Sign IMU result in Kinect video

In [197]:
#Sign IMU result in Kinect video
IMU_result_inKinect = []

IMU_rW_time = pd.to_datetime(IMU_rW_pd['timestamp'], format='%H:%M:%S:%f')
Kinect_rW_time = pd.to_datetime(Kinect_rW_pd['timestamp'], format='%H:%M:%S:%f')
i = 0
for Kinect_rW_timestamp in Kinect_rW_time:
    for si in range(i, len(IMU_rW_time)):
        if IMU_rW_time[si] <= Kinect_rW_timestamp and Kinect_rW_timestamp < IMU_rW_time[si+1]:
            IMU_result_inKinect.append(IMU_sample_rw_pred[i])
            i = si
            break
            
IMU_Kinect_sample_rw_pred = pd.DataFrame(np.array([IMU_result_inKinect, Kinect_sample_rw_pred]).T)

### Save to csv include result of predicting, headX_video, headY_video

In [198]:
result = pd.concat([IMU_Kinect_sample_rw_pred, Kinect_rW_pd[['Head_image_x', 'Head_image_y', 'timestamp']]], ignore_index=True, axis=1)

result.to_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/realWorldData/Kinect/ReachOutResult.csv', header=None, index=False)

# Next Step

- 測試真實的testing data看效果如何
- IMU加更多的feature..(切更細的mean(O), Correlation, Covariance..)
- Kinect加angle相關的atrribute, 並且座標zero點在自己身上,不會依據距離遠近而誤判
- Kinect & IMU feature 合併的話??不是個別判斷
- 程式部分
    - 將extract feature寫成function(O)
    - set sample/sec to var

DONE
- 實驗重作...(O)
- 程式部分
    - 將extract feature寫成function(O)
    - set sample/sec to var(O)

# Save to csv include percentage, headX_video, headY_video

In [ ]:
result = pd.concat([percentage, df[['HeadX_Video', 'HeadY_Video', 'TimeStamp']]], axis=1)

result.to_csv('C:\Research_temp\data\Result\ReachOutResult.csv', header=None, index=False)

In [ ]:
import glob
import os

path = r'D:\GoogleDrive\Graduate\Research\Research_HsinWei\Programs\data\IMUData\ReachOut'                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent


In [ ]:
for a, b in range(10), range(11):
    print(a, b)